## Do measurements

In [2]:
import logging
import traceback

from pprint import pprint
from clickhouse_driver import Client

from scripts.utils.file_utils import load_json, dump_json
from scripts.utils.helpers import haversine
from scripts.traceroute_measurements.traceroutes import serialize
from scripts.traceroute_measurements.three_tiers import tier_1, tier_2, tier_3
from default import ANCHORS_FILE, ANALYZABLE_FILE, DB_HOST


logging.basicConfig(level=logging.INFO)

### database for traceroutes

In [ ]:
query = (
        f"create table bgp_interdomain_te.street_lvl_traceroutes
        (
            src_addr String,
            dst_prefix String,
            dst_addr String,
            resp_addr String,
            proto Int16,
            hop Int16,
            rtt Float64,
            ttl Int16,
            prb_id Int64,
            msm_id Int64,
            tstamp Datetime('UTC')
        )"
        f"ENGINE = MergeTree()"
        f"order by (dst_addr, src_addr, tstamp)"
)

In [ ]:
    client = Client(DB_HOST)
    rows = client.execute(query)
    client.disconnect()

In [3]:
def get_all_info_geoloc(target_ip):
    logging.info(f"tier1 for {target_ip}")
    res = {'target_ip': target_ip, 'tier1:done': False, 'tier2:done': False,
           'tier3:done': False, 'negative_rtt_included': True}
    res = tier_1(target_ip, res)

    res['lat'] = res['tier1:lat']
    res['lon'] = res['tier1:lon']
    if res['tier1:lat'] == None or res['tier1:lon'] == None:
        return res
    res['tier1:done'] = True

    logging.info(f"tier2 for {target_ip}")
    res = tier_2(target_ip, res)

    if res['tier2:lat'] == None or res['tier2:lon'] == None:
        return res
    else:
        res['tier2:done'] = True
        res['lat'] = res['tier2:lat']
        res['lon'] = res['tier2:lon']

    logging.info(f"tier3 for {target_ip}")
    res = tier_3(target_ip, res)

    if res['tier3:lat'] != None and res['tier3:lon'] != None:
        res['tier3:done'] = True
        res['lat'] = res['tier3:lat']
        res['lon'] = res['tier3:lon']

    return res

### Main

In [ ]:
#anchors = load_json(ANCHORS_FILE)
anchors = load_json("anchors.json")
i = 0
try:
    #all_res = load_json(ANALYZABLE_FILE)
    all_res = load_json("all_res.json")
except FileNotFoundError:
    all_res = {}

for target in anchors:
    try:
        target_ip = target['address_v4']
        if target_ip in all_res:
            continue
        print(f"{i}:{target_ip}")
        i += 1
        lat = target['geometry']['coordinates'][1]
        lon = target['geometry']['coordinates'][0]
        res = {}
        res = get_all_info_geoloc(target_ip)
        res = serialize(res)
        res['lat_c'] = lat
        res['lon_c'] = lon

        if res['lat'] != None and res['lon'] != None:
            res['error'] = haversine(
                (res['lat'], res['lon']), (res['lat_c'], res['lon_c']))
        if 'tier1:lat' in res and 'tier1:lon' in res and res['tier1:lat'] != None and res['tier1:lon'] != None:
            res['tier1:error'] = haversine(
                (res['tier1:lat'], res['tier1:lon']), (res['lat_c'], res['lon_c']))
        if 'tier2:lat' in res and 'tier2:lon' in res and res['tier2:lat'] != None and res['tier2:lon'] != None:
            res['tier2:error'] = haversine(
                (res['tier2:lat'], res['tier2:lon']), (res['lat_c'], res['lon_c']))
        if 'tier3:lat' in res and 'tier3:lon' in res and res['tier3:lat'] != None and res['tier3:lon'] != None:
            res['tier3:error'] = haversine(
                (res['tier3:lat'], res['tier3:lon']), (res['lat_c'], res['lon_c']))

        all_res[target_ip] = res
    except Exception:
        traceback.print_exc()
        print(f"{target_ip} problem")
        pprint(res)

#dump_json(all_res, ANALYZABLE_FILE)
dump_json(all_res, "all_res.json")